# Deadly Visualizations!!!

![Image](../images/viz_types_portada.png)

## Setup

First we need to create a basic setup which includes:

- Importing the libraries.

- Reading the dataset file (source [Instituto Nacional de Estadística](https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259942408928&p=1259942408928&pagename=ProductosYServicios%2FPYSLayout)).

- Create a couple of columns and tables for the analysis.

__NOTE:__ some functions were already created in order to help you go through the challenge. However, feel free to perform any code you might need.

In [1]:
# imports

import sys
import re
sys.path.insert(0, "../modules")

import numpy as np
import pandas as pd

import plotly.express as px
import cufflinks as cf
cf.go_offline()
import plotly.graph_objs as go
import matplotlib as plt

import module as mod     # functions are include in module.py

In [2]:
# read dataset

deaths = pd.read_csv('../data/7947.csv', sep=';', thousands='.')

deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.5+ MB


In [3]:
# add some columns...you'll need them later

deaths['cause_code'] = deaths['Causa de muerte'].apply(mod.cause_code)
deaths['cause_group'] = deaths['Causa de muerte'].apply(mod.cause_types)
deaths['cause_name'] = deaths['Causa de muerte'].apply(mod.cause_name)

deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301158 entries, 0 to 301157
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Causa de muerte  301158 non-null  object
 1   Sexo             301158 non-null  object
 2   Edad             301158 non-null  object
 3   Periodo          301158 non-null  int64 
 4   Total            301158 non-null  int64 
 5   cause_code       301158 non-null  object
 6   cause_group      301158 non-null  object
 7   cause_name       301158 non-null  object
dtypes: int64(2), object(6)
memory usage: 18.4+ MB


In [4]:
# lets check the categorical variables

var_list = ['Sexo', 'Edad', 'Periodo', 'cause_code', 'cause_name', 'cause_group']

categories = mod.cat_var(deaths, var_list)
categories

,categorical_variable,number_of_possible_values,values
0,cause_code,117,"[001-102, 001-008, 001, 002, 003, 004, 005, 00..."
1,cause_name,117,"[I-XXII.Todas las causas, I.Enfermedades infec..."
2,Periodo,39,"[2018, 2017, 2016, 2015, 2014, 2013, 2012, 201..."
3,Edad,22,"[Todas las edades, Menos de 1 año, De 1 a 4 añ..."
4,Sexo,3,"[Total, Hombres, Mujeres]"
5,cause_group,2,"[Multiple causes, Single cause]"


In [5]:
# we need also to create a causes table for the analysis

causes_table = deaths[['cause_code', 'cause_name']].drop_duplicates().sort_values(by='cause_code').reset_index(drop=True)

causes_table

,cause_code,cause_name
0,001,Enfermedades infecciosas intestinales
1,001-008,I.Enfermedades infecciosas y parasitarias
2,001-102,I-XXII.Todas las causas
3,002,Tuberculosis y sus efectos tardíos
4,003,Enfermedad meningocócica
...,...,...
112,098,Suicidio y lesiones autoinfligidas
113,099,Agresiones (homicidio)
114,100,Eventos de intención no determinada
115,101,Complicaciones de la atención médica y quirúrgica


In [6]:
# And some space for free-style Pandas!!! (e.g.: df['column_name'].unique())
deaths['Causa de muerte'].unique()

array(['001-102  I-XXII.Todas las causas',
       '001-008  I.Enfermedades infecciosas y parasitarias',
       '001  Enfermedades infecciosas intestinales',
       '002  Tuberculosis y sus efectos tardíos',
       '003  Enfermedad meningocócica', '004  Septicemia',
       '005  Hepatitis vírica', '006  SIDA',
       '007  VIH+ (portador, evidencias de laboratorio del VIH, ...)',
       '008  Resto de enfermedades infecciosas y parasitarias y sus efectos tardíos',
       '009-041  II.Tumores',
       '009  Tumor maligno del labio, de la cavidad bucal y de la faringe',
       '010  Tumor maligno del esófago',
       '011  Tumor maligno del estómago', '012  Tumor maligno del colon',
       '013  Tumor maligno del recto, de la porción rectosigmoide y del ano',
       '014  Tumor maligno del hígado y vías biliares intrahepáticas',
       '015  Tumor maligno del páncreas',
       '016  Otros tumores malignos digestivos',
       '017  Tumor maligno de la laringe',
       '018  Tumor maligno d

## Lets make some transformations

Eventhough the dataset is pretty clean, the information is completely denormalized as you could see. For that matter a collection of methods (functions) are available in order to generate the tables you might need:

- `row_filter(df, cat_var, cat_values)` => Filter rows by any value or group of values in a categorical variable.

- `nrow_filter(df, cat_var, cat_values)` => The same but backwards. 

- `groupby_sum(df, group_vars, agg_var='Total', sort_var='Total')` => Add deaths by a certain variable.

- `pivot_table(df, col, x_axis, value='Total')`=> Make some pivot tables, you might need them...

__NOTE:__ be aware that the filtering methods can perform a filter at a time. Feel free to perform the filter you need in any way you want or feel confortable with.

In [ ]:
#cause_code_dataset = mod.row_filter(deaths, 'cause_group', ['Single cause'])
#cause_code_dataset

In [7]:
# Example 1

dataset = mod.row_filter(deaths, 'Sexo', ['Hombres','Mujeres'])
dataset = mod.nrow_filter(dataset, 'Edad', ['Todas las edades'])


print(dataset.head())
dataset.info()

                    Causa de muerte     Sexo               Edad  Periodo  \
0  001-102  I-XXII.Todas las causas  Mujeres  De 85 a 89 años       2018   
1  001-102  I-XXII.Todas las causas  Mujeres  De 85 a 89 años       2017   
2  001-102  I-XXII.Todas las causas  Mujeres  De 85 a 89 años       2015   
3  001-102  I-XXII.Todas las causas  Mujeres  De 85 a 89 años       2016   
4  001-102  I-XXII.Todas las causas  Mujeres  De 85 a 89 años       2012   

   Total cause_code      cause_group               cause_name  
0  51373    001-102  Multiple causes  I-XXII.Todas las causas  
1  50929    001-102  Multiple causes  I-XXII.Todas las causas  
2  50832    001-102  Multiple causes  I-XXII.Todas las causas  
3  48674    001-102  Multiple causes  I-XXII.Todas las causas  
4  47761    001-102  Multiple causes  I-XXII.Todas las causas  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191646 entries, 0 to 191645
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---

In [8]:
dataset

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
0,001-102 I-XXII.Todas las causas,Mujeres,De 85 a 89 años,2018,51373,001-102,Multiple causes,I-XXII.Todas las causas
1,001-102 I-XXII.Todas las causas,Mujeres,De 85 a 89 años,2017,50929,001-102,Multiple causes,I-XXII.Todas las causas
2,001-102 I-XXII.Todas las causas,Mujeres,De 85 a 89 años,2015,50832,001-102,Multiple causes,I-XXII.Todas las causas
3,001-102 I-XXII.Todas las causas,Mujeres,De 85 a 89 años,2016,48674,001-102,Multiple causes,I-XXII.Todas las causas
4,001-102 I-XXII.Todas las causas,Mujeres,De 85 a 89 años,2012,47761,001-102,Multiple causes,I-XXII.Todas las causas
...,...,...,...,...,...,...,...,...
191641,069 Enteritis y colitis no infecciosas,Mujeres,Menos de 1 año,1992,0,069,Single cause,Enteritis y colitis no infecciosas
191642,069 Enteritis y colitis no infecciosas,Mujeres,Menos de 1 año,2007,0,069,Single cause,Enteritis y colitis no infecciosas
191643,069 Enteritis y colitis no infecciosas,Mujeres,Menos de 1 año,2006,0,069,Single cause,Enteritis y colitis no infecciosas
191644,069 Enteritis y colitis no infecciosas,Mujeres,Menos de 1 año,2005,0,069,Single cause,Enteritis y colitis no infecciosas


In [9]:
dataset[dataset['Causa de muerte'] == '002  Tuberculosis y sus efectos tardíos']

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
27960,002 Tuberculosis y sus efectos tardíos,Hombres,De 65 a 69 años,1981,169,002,Single cause,Tuberculosis y sus efectos tardíos
29864,002 Tuberculosis y sus efectos tardíos,Hombres,De 75 a 79 años,1981,153,002,Single cause,Tuberculosis y sus efectos tardíos
30339,002 Tuberculosis y sus efectos tardíos,Hombres,De 70 a 74 años,1980,149,002,Single cause,Tuberculosis y sus efectos tardíos
30547,002 Tuberculosis y sus efectos tardíos,Hombres,De 55 a 59 años,1980,147,002,Single cause,Tuberculosis y sus efectos tardíos
30569,002 Tuberculosis y sus efectos tardíos,Hombres,De 65 a 69 años,1980,147,002,Single cause,Tuberculosis y sus efectos tardíos
...,...,...,...,...,...,...,...,...
178194,002 Tuberculosis y sus efectos tardíos,Hombres,De 5 a 9 años,2003,0,002,Single cause,Tuberculosis y sus efectos tardíos
178195,002 Tuberculosis y sus efectos tardíos,Hombres,De 5 a 9 años,2001,0,002,Single cause,Tuberculosis y sus efectos tardíos
178197,002 Tuberculosis y sus efectos tardíos,Hombres,De 5 a 9 años,2000,0,002,Single cause,Tuberculosis y sus efectos tardíos
178270,002 Tuberculosis y sus efectos tardíos,Hombres,De 1 a 4 años,2013,0,002,Single cause,Tuberculosis y sus efectos tardíos


In [10]:
tuberculosis = dataset[dataset['Causa de muerte'] == '002  Tuberculosis y sus efectos tardíos'].sort_values('Periodo')
tuberculosis.head()

,Causa de muerte,Sexo,Edad,Periodo,Total,cause_code,cause_group,cause_name
36573,002 Tuberculosis y sus efectos tardíos,Hombres,De 50 a 54 años,1980,103,002,Single cause,Tuberculosis y sus efectos tardíos
60917,002 Tuberculosis y sus efectos tardíos,Hombres,De 85 a 89 años,1980,31,002,Single cause,Tuberculosis y sus efectos tardíos
52524,002 Tuberculosis y sus efectos tardíos,Mujeres,De 65 a 69 años,1980,46,002,Single cause,Tuberculosis y sus efectos tardíos
126171,002 Tuberculosis y sus efectos tardíos,Hombres,De 1 a 4 años,1980,2,002,Single cause,Tuberculosis y sus efectos tardíos
136686,002 Tuberculosis y sus efectos tardíos,Hombres,De 20 a 24 años,1980,1,002,Single cause,Tuberculosis y sus efectos tardíos


In [11]:
tuberculosis_hist = tuberculosis.groupby('Periodo')[['Total']].sum().reset_index()
tuberculosis_hist

,Periodo,Total
0,1980,1476
1,1981,1583
2,1982,1338
3,1983,1288
4,1984,1141
5,1985,1126
6,1986,1071
7,1987,1062
8,1988,999
9,1989,943


In [12]:
#Evolución de muertes por tuberculosis

fig = go.Figure(data=go.Scatter(x=tuberculosis_hist['Periodo'].astype(dtype=str), 
                        y=tuberculosis_hist['Total'],
                        marker_color='indianred', text="counts"))
fig.update_layout({"title": 'Tuberculosis',
                   "xaxis": {"title":"Años"},
                   "yaxis": {"title":"Muertes Totales"},
                   "showlegend": False})
plt.yticks(range(0,1600))
fig.show()

AttributeError: module 'matplotlib' has no attribute 'yticks'

In [ ]:
#Inicio del cálculo mayor causa de la muerte histórica
#Filtro eliminando el subconjunto de muertes totales

dataset_totales = mod.row_filter(deaths, 'Sexo', ['Hombres','Mujeres'])
dataset_tot = mod.nrow_filter(dataset_totales, 'Edad', ['Todas las edades'])
dataset_tot = mod.nrow_filter(dataset_tot, 'Causa de muerte', ['001-102  I-XXII.Todas las causas'])

print(dataset_tot.head())
dataset_tot.info()

In [ ]:
#Reviso la información que me devuelve el dataframe
dataset_tot[:5]

In [ ]:
#Genero un nuevo dataframe sumando el total de muerte por enfermedad sin importar el año
max_muertes = dataset_tot.groupby('Causa de muerte')[['Total']].sum().sort_values(by='Total',ascending=False).reset_index()
max_muertes_500 = max_muertes.loc[max_muertes['Total'] >= 500000]
max_muertes_500

In [ ]:
#Gráfica de barras con las 10 mayores causas de muerte de la historia

fig_hist = px.bar(max_muertes_500, x='Causa de muerte', y='Total')
fig_hist.show()

In [ ]:
#Sabiendo que las enfermedades del sistema circulatorio sobrepasan cualquier otra (cerca tenemos tumores)
#Vamos a ver la evolución anual por género

sist_circulatorio = dataset[dataset['Causa de muerte'] == '053-061 IX.Enfermedades del sistema circulatorio'].sort_values('Periodo')
sist_circulatorio.reset_index(inplace=True)
sist_circulatorio.head()

In [ ]:
#Creo un nuevo df solo con los campos que necesito: sexo, edad y total de muertes por enfermedad del sist. circulatorio
#Agrupo el total de muertes por año y por sexo

sist_circulatorio_hist = sist_circulatorio[['Sexo', 'Periodo', 'Total']].copy()
sis_circ_total = sist_circulatorio_hist.groupby(['Periodo','Sexo'])[['Total']].sum().reset_index()
sis_circ_total


In [ ]:
#Creo una gráfica de lineas separando por sexo

fig_sc = px.line(sis_circ_total, x="Periodo", y="Total", color='Sexo')
fig_sc.show()

In [ ]:
sist_circulatorio
sist_circulatorio_hist = sist_circulatorio.groupby('Periodo')[['Total']].sum().reset_index()
sist_circulatorio_hist

In [ ]:
#Evolución de muertes Enfermedades del sistema circulatorio

fig_sis = go.Figure(data=go.Scatter(x=sist_circulatorio_hist['Periodo'].astype(dtype=str), 
                        y=sist_circulatorio_hist['Total'],
                        marker_color='indianred', text="counts"))
fig_sis.update_layout({"title": 'Enfermedades Sistema Circulatorio',
                   "xaxis": {"title":"Años"},
                   "yaxis": {"title":"Muertes Totales"},
                   "showlegend": False})
fig_sis.show()

In [ ]:
#Aprovecho el dataframe filtrado por total de muertes de la historia (sin tener en cuenta el año)
#De las 10 últimas ya veo que la que menos me debería preocupar es la muerte subita infantil
#Y en segundo lugar la Enfermedad meningocócica
min_muertes = max_muertes.loc[max_muertes['Total'] <= 5000]
min_muertes

In [ ]:
#Falta pintar un gráfico - Pinto uno que compare este con el 

In [ ]:

  
X = ['Group A','Group B','Group C','Group D']
Ygirls = [10,20,20,40]
Zboys = [20,30,25,30]
  
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, Ygirls, 0.4, label = 'Girls')
plt.bar(X_axis + 0.2, Zboys, 0.4, label = 'Boys')
  
plt.xticks(X_axis, X)
plt.xlabel("Groups")
plt.ylabel("Number of Students")
plt.title("Number of Students in each group")
plt.legend()
plt.show()

In [ ]:
# Example 2

group = ['cause_code','Periodo']
dataset = mod.groupby_sum(deaths, group)
dataset.head()


In [ ]:
# Example 3

dataset = mod.pivot_table(dataset, 'cause_code', 'Periodo')
dataset.head()


## ...and finally, show me some insights with Plotly!!!

In [ ]:
# Cufflinks histogram
'''
dataset_column.iplot(kind='hist',
                     title='VIZ TITLE',
                     yTitle='AXIS TITLE',
                     xTitle='AXIS TITLE')
'''

In [ ]:
# Cufflinks bar plot
'''
dataset_bar.iplot(kind='bar',
                  x='VARIABLE',
                  xTitle='AXIS TITLE',
                  yTitle='AXIS TITLE',
                  title='VIZ TITLE')
'''

In [ ]:
# Cufflinks line plot
'''
dataset_line.iplot(kind='line',
                   x='VARIABLE',
                   xTitle='AXIS TITLE',
                   yTitle='AXIS TITLE',
                   title='VIZ TITLE')
'''

In [ ]:
# Cufflinks scatter plot
'''
dataset_scatter.iplot(x='VARIABLE', 
                      y='VARIABLE', 
                      categories='VARIABLE',
                      xTitle='AXIS TITLE', 
                      yTitle='AXIS TITLE',
                      title='VIZ TITLE')
'''